In [ ]:
import json
import pickle
import numpy as np
from inference.utils.vision_process import fetch_video

In [ ]:
data = json.load(open("/home/atuin/g102ea/shared/group_10/datasets/charades/subset_charades_mcq_with_time.json", "r"))
masks = pickle.load(open("/home/atuin/g102ea/shared/group_10/results/analysis/charades/charades_tb_8_1%_0.5/masks.json", "rb"))

In [ ]:
def mask_relevance(mask, oracle):
    return float(mask[oracle[0]:oracle[1], ...].sum()/mask.sum())

In [ ]:
def get_random_mask(shape, budget):
    array = np.zeros(shape, dtype=bool)
    total_elements = np.prod(shape)
    indices = np.random.choice(total_elements, budget, replace=False)
    np.put(array, indices, True)
    return array

In [ ]:
uniform_patch_accuracy = []
patch_accuracy = []
oracle = []
for i in range(len(masks)):
    mask = np.array(masks[i])
    video_length = data["mcq_data"]["video_lengths"][i]
    timestamps = data["mcq_data"]["mcq_timestamps"][i]
    timestamps = (max(timestamps[0], 0), min(timestamps[1], video_length))
    oracle_frames = (int(max(0, timestamps[0]/video_length * mask.shape[0])), int(min(mask.shape[0], np.ceil(timestamps[1]/video_length * mask.shape[0]))))
    patch_accuracy.append(mask_relevance(mask, oracle_frames))
    random_mask = get_random_mask(mask.shape, mask.sum())
    uniform_patch_accuracy.append(mask_relevance(random_mask, oracle_frames))
    oracle.append((oracle_frames, mask.shape[0]))

In [ ]:
print("Our avg patch accuracy = ", sum(patch_accuracy)/len(patch_accuracy))
print("Uniform avg patch accuracy = ", sum(uniform_patch_accuracy)/len(uniform_patch_accuracy))

In [ ]:
sum([1 for i in oracle if (i[0][1] - i[0][0])/i[1] > 0.35])